# DIY Covid-19 Dashboard

The grapgh below displays data aquired from Public Health England (PHE), regarding new COVID-19 cases, hospital admissions and occupied beds with mechanical ventilators.
To display the data of intrest, select the corrsponding option in the selection box on the top left of the graph. To display multiple datasets on the graph, hold down the "Ctrl" and click on the relevent option fields.
In order to modifiy the scale of the Y-axis, select the approperiate option on the top right of the graph. To display a linear Y-axis select the "linear" option, and to display a logorithmic Y-axis select the "log" option.
To update the graph with the most up to date information aviable from PHE, click on the "Refersh Data" button.


In [1]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

%matplotlib inline
plt.rcParams['figure.dpi'] = 150

In [2]:
with open("newcases.json", "rt") as FILE:
    info=json.load(FILE)

In [3]:
def parse_date(datestring):
    
    return pd.to_datetime(datestring, format="%Y-%m-%d")
def wrangle_data(rawdata):
    
    infolist=info['data']

    day=[dictionary['date'] for dictionary in infolist ]
    day.sort()
    
    startdate=parse_date(day[0])
    enddate=parse_date(day[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    newcaseseriesdf=pd.DataFrame(index=index, columns=['Cases', "Admissions", "OccupiedMVBeds"])

    for entry in infolist: 
        date=parse_date(entry['date'])
        for column in ['Cases', "Admissions", "OccupiedMVBeds"]:
            if pd.isna(newcaseseriesdf.loc[date, column]): 
                value= float(entry[column]) if entry[column]!=None else 0.0
                newcaseseriesdf.loc[date, column]=value
    return newcaseseriesdf

newcaseseriesdf=wrangle_data(info) 

In [4]:


def access_api():
    new_covid_cases = {
    'date' : 'date',
    'Cases' : 'newCasesByPublishDate',
    'Admissions' : 'newAdmissions',
    'OccupiedMVBeds' : 'covidOccupiedMVBeds' 
    }

    within_england = ['areaType=nation', 'areaName=England']

    api = Cov19API(filters=within_england, structure=new_covid_cases)
    newcases=api.get_json()

    with open("newcases.json", "wt") as FILE:
        json.dump(newcases, FILE)
        
    print("I'm downloading data from the API...")
    print("...all done.")
    return newcases 


def api_button_callback(button):
    apidata=access_api()
    newcaseseriesdf=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='', 
    tooltip='Click to download current Public Health England data',
    icon='download'
)


apibutton.on_click(api_button_callback) 
display(apibutton)



Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

In [5]:



cases_cols=wdg.SelectMultiple(
    options=['Cases', 'Admissions', 'OccupiedMVBeds'], 
    value=['Cases'], 
    rows=3, 
    description='Select',
    disabled=False)




case_scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)


con=wdg.HBox([cases_cols, case_scale])


def case_graph(casecolumns, casescale):
    if casescale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(casecolumns)
    if ncols>0:
         newcaseseriesdf[list(casecolumns)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
def refresh_graph():
    current=case_scale.value
    if current==case_scale.options[0]:
        other=case_scale.options[1]
    else:
        other=case_scale.options[0]
    case_scale.value=other # forces the redraw
    case_scale.value=current # now we can change it back
    
outputs=wdg.interactive_output(case_graph, {'casecolumns': cases_cols, "casescale": case_scale})
display(con, outputs)



Output()